In [1]:
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 200)

In [2]:
df = pl.read_csv("/home/surayuth/her2/extracted_features/linefeat|scale_0.25|minhole_10|maxhole_1000|maxres_0.25.csv")

In [11]:
def color_rows(row):
    if row['label'] == 0:
        return ['background-color: darkgreen'] * len(row)
    else:
        return ['background-color: darkred'] * len(row)

df \
    .group_by("case") \
    .agg(
        pl.len().alias("count"),
        pl.when(pl.col("deg") == 0).then(pl.lit(1)).sum().alias("num0"),
        pl.when(pl.col("deg") == 1).then(pl.lit(1)).sum().alias("num1"),
        pl.when(pl.col("deg") > 1).then(pl.lit(1)).sum().alias("num>1"),
        pl.col("n").mean(),
        pl.when(pl.col("deg") == 1).then(pl.col("param_v")).mean().alias("avg_param_v1"),
        pl.col("mean_y").mean(),
        pl.col("ihc_score").first(),
        pl.col("label").first()
    ) \
    .sort("ihc_score", "avg_param_v1", "case") \
    .filter(
        pl.col("ihc_score").is_in(["2-", "2+"])
    ) \
    .sort("ihc_score", descending=True) \
    .to_pandas() \
    .style.apply(color_rows, axis=1)

,case,count,num0,num1,num>1,n,avg_param_v1,mean_y,ihc_score,label
0,17 Jan 23 HER2 2+ Dish - with Control,18,0,0,18,28.888889,nan,-4.919535,2-,0
1,8 Nov HER2 2+ DISH - case 1,59,0,0,59,35.372881,nan,-4.423744,2-,0
2,HER2 2+ DISH- case 06,94,94,0,0,0.000000,nan,0.000000,2-,0
3,HER2 2+ DISH- case 04,49,0,7,42,33.326531,9.263654,-3.854093,2-,0
4,16 Aug 22 Case 3 HER2 2+ DISH -,42,0,21,21,17.190476,12.435671,-3.454986,2-,0
5,HER2 2+ DISH- case 03,32,0,12,20,21.437500,12.865398,-3.860313,2-,0
6,1 Sep HER2 2+ DISH - case 1,49,0,20,29,18.734694,16.277940,-3.940852,2-,0
7,22 Nov HER2 2+ DISH -,122,1,51,70,20.139344,16.552861,-3.941680,2-,0
8,HER2 2+ DISH-,75,0,30,45,22.800000,16.876436,-4.337363,2-,0
9,1 Sep HER2 2+ DISH - case 2,64,0,31,33,20.687500,17.139702,-4.084332,2-,0
